In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Reshape for CNN (batch, 28, 28, 1)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Train
model.fit(X_train, y_train, epochs=5, validation_split=0.1)

# Evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print("Test Accuracy:", acc)

# Save
model.save("mnist_cnn.h5")


Epoch 1/5
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1386 - accuracy: 0.9580 - val_loss: 0.0506 - val_accuracy: 0.9860
Epoch 2/5
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0448 - accuracy: 0.9861 - val_loss: 0.0379 - val_accuracy: 0.9887
Epoch 3/5
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0304 - accuracy: 0.9904 - val_loss: 0.0361 - val_accuracy: 0.9892
Epoch 4/5
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0235 - accuracy: 0.9926 - val_loss: 0.0306 - val_accuracy: 0.9918
Epoch 5/5
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0159 - accuracy: 0.9946 - val_loss: 0.0368 - val_accuracy: 0.9910
313/313 - 1s - loss: 0.0334 - accuracy: 0.9902 - 993ms/epoch - 3ms/step
Test Accuracy: 0.9901999831199646


In [ ]:
import numpy as np
import gradio as gr
from PIL import Image

def predict_digit(img):
    #Convert to grayscale
    img = Image.fromarray(img).convert("L")
    
    #Invert colors 
    img = np.invert(np.array(img))
    
    #Resize to 28x28
    img = Image.fromarray(img).resize((28, 28))
    
    # Normalize and add batch dimension
    img = np.array(img).astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)
    
    #Predict
    prediction = model.predict(img)
    return {str(i): float(prediction[0][i]) for i in range(10)}

demo = gr.Interface(
    fn=predict_digit,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Label(num_top_classes=3),
    title="MNIST Digit Classifier",
    description="Upload the image of any digit from (0-9), and let the model predict it."
)

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


1/1 [==============================] - 0s 394ms/step
